# Project: Road Accidents in France based on Annual Road Traffic Accident Injury Database (2005 - 2023)

## Step 1: Exploration + Data Visualization

---

### Description

For each accident involving injury (i.e., an accident occurring on a road open to public traffic, involving at least one vehicle and resulting in at least one victim requiring medical treatment), information describing the accident is recorded by the law enforcement unit (police, gendarmerie, etc.) that responded to the scene. These entries are compiled into a form called the *Injury Accident Analysis Bulletin*. Together, these forms constitute the national road traffic accident file, known as the **BAAC File**, administered by the **National Interministerial Road Safety Observatory (ONISR)**.

The databases, extracted from the BAAC file, list all road traffic accidents involving injuries that occurred during a given year in:

- Mainland France
- Overseas departments (Guadeloupe, French Guiana, Martinique, Réunion, and Mayotte since 2012)
- Overseas territories (Saint-Pierre and Miquelon, Saint-Barthélemy, Saint-Martin, Wallis and Futuna, French Polynesia, and New Caledonia; available since 2019)

The databases from 2005 to 2023 are structured annually and consist of four CSV files:

- **Caractéristiques** (Accident details)
- **Lieux** (Locations)
- **Véhicules** (Vehicles)
- **Usagers** (Users)

---

The objective of this project is to try to predict the severity of road accidents in France. Predictions will be based on historical data.
A first step is to study and apply methods to clean the dataset. Once the dataset is clean, a second step is to extract from the match history the characteristics that seem to be relevant for estimating the severity of accidents. Then, from its results, the objective is to work on a scoring of the risk zones according to the meteorological information, the geographical location (GPS coordinates, satellite images, …).
Once the model has been trained, we will compare our model with historical data.

---

## Important Notes

- **Runaway Users:**  
  Since 2021, data on runaway users has been added.  
  This results in missing information such as sex, age, and injury severity (unharmed, slightly injured, hospitalized).

- **Missing Data:**  
  Most variables across the four main files may contain:
  - Empty cells
  - Zeros
  - Periods (`.`)
In these cases, the field was either not populated by law enforcement or the information was deemed irrelevant.
Some categories use -1 to indicate 'not specified'.

- **Hospitalized Injured Persons:**  
  - Data regarding the classification of hospitalized injured persons since 2018 cannot be compared with previous years due to changes in the law enforcement data entry process. The "hospitalized injured person" indicator has not been certified by the Public Statistics Authority since 2019.

---

## Data Handling

I downloaded the data from:  
[Data Source - data.gouv.fr](https://www.data.gouv.fr/en/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/)  
and saved it into the `data/raw/` folder.

Each year includes four main tables:

- **Caractéristiques** (Caracteristics, or Accident details)
- **Lieux** (Locations)
- **Véhicules** (Vehicles)
- **Usagers** (Users)

Additionally, data on **registered vehicles (vehicules-immatricules-baac)** for 2009–2022 is available.  
However, I chose not to use this dataset for now, as it may introduce problems in machine learning algorithms due to missing years.

To streamline data processing, I corrected the names of some downloaded files to follow the format:  
`category_year` (e.g., `Caracteristiques_2020.csv`, `Vehicules_2018.csv`).

---

The file columns contain encoded information about various accident characteristics.  
The descriptions are provided in the latest version of the encoding documentation:

- **description-des-bases-de-donnees-annuelles.pdf** (description of annual databases)
- **Caracteristics.docx** (translated characteristics from French to English)

Both documents are stored in the `references/` folder.

Short description of the columns 
| Column                | Description |
|-----------------------|-----------------------------------|
| **num_acc**           | Unique identifier of the accident, assigned by the law enforcement. Join key between the files characteristics, locations, vehicles and users. |
| **an**                | Year of the accident. |
| **mois**              | Month of the accident. |
| **jour**              | Day of the accident. |
| **hrmn**              | Hour and minutes of the accident (format hhmm). |
| **lum**               | Lighting conditions at the time of the accident: 1 = Full daylight, 2 = Twilight or dawn, 3 = Night without public lighting, 4 = Night with public lighting not lit, 5 = Night with public lighting lit. |
| **agg**               | Urban area indicator: 1 = Outside urban area, 2 = Inside urban area. |
| **int**               | Type of intersection: 1 = Outside intersection, 2 = Intersection in X, 3 = Intersection in T, 4 = Intersection in Y, 5 = More than 4 branches, 6 = Roundabout, 7 = Square, 8 = Level crossing, 9 = Other intersection. |
| **atm**               | Atmospheric conditions at the time of the accident: -1 = Not specified, 1 = Normal, 2 = Light rain, 3 = Heavy rain, 4 = Snow–hail, 5 = Fog–smoke, 6 = Strong wind–storm, 7 = Dazzling, 8 = Overcast, 9 = Other. |
| **col**               | Type of collision: -1 = Not specified, 1 = Two vehicles – head-on, 2 = Two vehicles – rear-end, 3 = Two vehicles – side, 4 = Three vehicles or more – in chain, 5 = Three vehicles or more – multiple collisions, 6 = Other collision, 7 = Without collision. |
| **com**               | INSEE code of the municipality of the accident. |
| **adr**               | Postal address of the accident, when the accident is located inside urban area. |
| **gps**               | GPS coordinates (raw text). |
| **lat**               | Latitude (decimal degrees). |
| **long**              | Longitude (decimal degrees). |
| **dep**               | INSEE department code of the accident. |
| **catr**              | Road category: 1 = Highway, 2 = National road, 3 = Departmental road, 4 = Communal road, 5 = Outside public network, 6 = Parking lot, 7 = Urban metropolitan road, 9 = Other. |
| **voie**              | Name or number of the road at the place of the accident. |
| **v1**                | Numeric sub-identifier for the road (e.g., "2 bis" expressed as number). |
| **v2**                | Alphanumeric sub-identifier for the road. |
| **circ**              | Traffic regime: -1 = Not specified, 1 = One way, 2 = Two way, 3 = Separated carriageways, 4 = Variable. |
| **nbv**               | Number of lanes for vehicles at the section of the road where the accident happened. |
| **pr**                | PR number (reference point, upstream terminal) — -1 if not filled. |
| **pr1**               | Distance to PR in meters (relative to the upstream terminal) — -1 if not filled. |
| **vosp**              | Reserved lane: -1 = Not specified, 0 = Not applicable, 1 = Cycle track, 2 = Cycle lane, 3 = Reserved lane. |
| **prof**              | Longitudinal profile of the road: -1 = Not specified, 1 = Flat, 2 = Slope, 3 = Top of hill, 4 = Bottom of hill. |
| **plan**              | Road plan: -1 = Not specified, 1 = Straight, 2 = Left curve, 3 = Right curve, 4 = "S" curve. |
| **lartpc**            | Width of the central reservation (in meters). |
| **larrout**           | Width of the carriageway assigned to vehicles (in meters). |
| **surf**              | Surface condition: -1 = Not specified, 1 = Normal, 2 = Wet, 3 = Puddles, 4 = Flooded, 5 = Snow, 6 = Mud, 7 = Icy, 8 = Fatty/oily, 9 = Other. |
| **infra**             | Infrastructure: -1 = Not specified, 0 = None, 1 = Tunnel, 2 = Bridge, 3 = Interchange or ramp, 4 = Railway, 5 = Crossing, 6 = Pedestrian area, 7 = Toll zone, 8 = Construction, 9 = Other. |
| **situ**              | Situation: -1 = Not specified, 0 = None, 1 = On roadway, 2 = On emergency lane, 3 = On shoulder, 4 = On sidewalk, 5 = On cycle path, 6 = On special route, 8 = Other. |
| **env1**              | Environment at the site of the accident (additional codings, see documentation if available). |
| **senc**              | Direction of movement: -1 = Not specified, 0 = Unknown, 1 = Ascending, 2 = Descending, 3 = No reference. |
| **catv**              | Category of vehicle involved: 01 = Bicycle, 07 = Passenger car, 13 = Heavy truck, 31 = Motorcycle, 37 = Bus, etc. (see full coding in documentation). |
| **occutc**            | Number of people present in the vehicle (for public transport vehicles). |
| **obs**               | Fixed obstacle hit: -1 = Not specified, 0 = Not applicable, 1 = Parked vehicle, 2 = Tree, 3 = Post, 4 = Rail guard, 5 = Concrete wall, 6 = Building, 7 = Fire hydrant, 8 = Lamp post, 9 = Other. |
| **obsm**              | Mobile obstacle hit: -1 = Not specified, 0 = None, 1 = Pedestrian, 2 = Vehicle, 3 = Animal, 4 = Other. |
| **choc**              | Initial point of impact: -1 = Not specified, 0 = None, 1 = Front, 2 = Front right, 3 = Front left, 4 = Rear, 5 = Rear right, 6 = Rear left, 7 = Side right, 8 = Side left. |
| **manv**              | Main maneuver before the accident: -1 = Not specified, 1 = No change, 2 = Stopped, 3 = Stationary, 4 = Reversing, 5 = Parking, 6 = Starting, 7 = Overtaking right, 8 = Overtaking left, 9 = Changing lanes, 10 = U-turn, 11 = Turning right, 12 = Turning left, 13 = Other. |
| **num_veh**           | Vehicle identifier in the accident, allows linking with occupants and users. Alphanumeric code. |
| **place**             | Position occupied by the user in the vehicle or accident: 10 = Pedestrian (see documentation for other positions/seats). |
| **catu**              | Category of user: 1 = Driver, 2 = Passenger, 3 = Pedestrian. |
| **grav**              | Severity of injury for the user: 1 = Unharmed, 2 = Killed, 3 = Hospitalized, 4 = Light injury. |
| **sexe**              | Gender of the user: 1 = Male, 2 = Female. |
| **trajet**            | Reason for the journey: -1 or 0 = Not specified, 1 = Home-work, 2 = Home-school, 3 = Shopping, 4 = Professional, 5 = Leisure, 9 = Other. |
| **secu**              | Safety equipment used by the user. Up to 2018: 0 = None, 1 = Seat belt, 2 = Helmet, 3 = Child seat, 4 = Reflective vest, 5 = Other (see secu1/secu2/secu3 for 2019+). |
| **locp**              | Location of the pedestrian: -1 = Not specified, 1 = On roadway, 2 = On shoulder, 3 = On crosswalk, 4 = On cycle path, 5 = On sidewalk, 6 = On special route, 7 = Other. |
| **actp**              | Action of the pedestrian: -1 = Not specified, 0 = Not applicable, 1 = Heading toward the vehicle, 2 = Moving away, 3 = Crossing, 4 = Waiting, 5 = Playing, 6 = Working, 9 = Other. |
| **etatp**             | Pedestrian's situation: -1 = Not specified, 1 = Alone, 2 = Accompanied, 3 = In group. |
| **an_nais**           | Year of birth of the user involved in the accident. |
| **num_veh_usag**      | Vehicle identifier (user table), alphanumeric code. |
| **vma**               | Maximum speed limit at the location and time of the accident (in km/h). |
| **id_vehicule**       | Unique identifier of the vehicle (in users and vehicles files). |
| **motor**             | Engine type: -1 = Not specified, 1 = Internal combustion, 2 = Hybrid, 3 = Electric, 4 = Hydrogen, 5 = Other. |
| **id_vehicule_usag**  | Unique vehicle identifier from the users file. |
| **secu1**             | 1st safety equipment used (from 2019 onwards). |
| **secu2**             | 2nd safety equipment used (from 2019 onwards). |
| **secu3**             | 3rd safety equipment used (from 2019 onwards). |
| **id_usager**         | Unique identifier of the user (in the users file). |



---

To prepare the data for machine learning, all the individual files must be merged into a single dataset. The Num_Acc column acts as the main key, allowing us to join the Caractéristiques, Lieux, Véhicules, and Usagers tables for each accident record.

In [1]:
#Load the packages 
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_classif

In [3]:
raw_path = "../data/raw"
processed_path = "../data/processed"
output_file = os.path.join(processed_path, "accidents_merged_2005_2023.csv")

#Some files have different delimiter and a typical French encoder ISO-8859-1
def robust_read_csv(fpath, encoding="ISO-8859-1"): 
    for delim in [",", ";", "\t"]:
        df = pd.read_csv(fpath, delimiter=delim, encoding=encoding, on_bad_lines="skip", low_memory=False)
        df.columns = df.columns.str.strip().str.lower().str.replace('"', '')
        # Rename for accident_id
        if 'accident_id' in df.columns:
            df = df.rename(columns={'accident_id': 'num_acc'}) #some datasets had different name for the num_acc key column
        if 'num_acc' in df.columns:
            return df
    print(f"!! Could not properly split columns for {fpath}, got: {df.columns.tolist()}")
    return df

all_years = []
for year in range(2005, 2024):
    files = {
        "carac": os.path.join(raw_path, f"caracteristiques_{year}.csv"),
        "lieux": os.path.join(raw_path, f"lieux_{year}.csv"),
        "vehic": os.path.join(raw_path, f"vehicules_{year}.csv"),
        "usag": os.path.join(raw_path, f"usagers_{year}.csv"),
    }
    dfs = {k: robust_read_csv(fpath) for k, fpath in files.items()}

    if all('num_acc' in df.columns for df in dfs.values()):
        df = dfs["carac"].merge(dfs["lieux"], on="num_acc", how="inner", suffixes=('', '_lieux')) #Keep only rows that have a match in both DataFrames
        df = df.merge(dfs["vehic"], on="num_acc", how="inner", suffixes=('', '_vehic'))
        df = df.merge(dfs["usag"], on="num_acc", how="inner", suffixes=('', '_usag'))
        df["an"] = year  # Replace 'an' column with the current year for unification, for some years it was 5 instead of 2005
        all_years.append(df)
    else:
        print(f"Skipping year {year}: 'num_acc' not found in all files.")

# Combine all years and save
merged_df = pd.concat(all_years, ignore_index=True)
os.makedirs(processed_path, exist_ok=True)
merged_df.to_csv(output_file, index=False)
print(f"Merged file saved as: {output_file}")


Merged file saved as: ../data/processed\accidents_merged_2005_2023.csv


After a big merge like this, we need have a quick data health check.

In [5]:
display(merged_df.shape)
display(merged_df.head(10))
#For the better understanding, I want to look at all columns, so I extract 100 random rows as a separate file
#df = pd.read_csv("../data/processed/accidents_merged_2005_2023.csv", low_memory=False)
#df_sample = df.sample(n=100, random_state=42)
#df_sample.to_csv("../data/processed/accidents_sample_100_random.csv", index=False)

(5314184, 60)

,num_acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,an_nais,num_veh_usag,vma,id_vehicule,motor,id_vehicule_usag,secu1,secu2,secu3,id_usager
0,200500000001,2005,1,12,1900,3,2,1,1.0,3.0,...,1976.0,A01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200500000001,2005,1,12,1900,3,2,1,1.0,3.0,...,1968.0,B02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200500000001,2005,1,12,1900,3,2,1,1.0,3.0,...,1964.0,B02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200500000001,2005,1,12,1900,3,2,1,1.0,3.0,...,2004.0,B02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200500000001,2005,1,12,1900,3,2,1,1.0,3.0,...,1998.0,B02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,200500000001,2005,1,12,1900,3,2,1,1.0,3.0,...,1991.0,B02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,200500000001,2005,1,12,1900,3,2,1,1.0,3.0,...,1976.0,A01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,200500000001,2005,1,12,1900,3,2,1,1.0,3.0,...,1968.0,B02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,200500000001,2005,1,12,1900,3,2,1,1.0,3.0,...,1964.0,B02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,200500000001,2005,1,12,1900,3,2,1,1.0,3.0,...,2004.0,B02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5314184 entries, 0 to 5314183
Data columns (total 60 columns):
 #   Column            Dtype  
---  ------            -----  
 0   num_acc           int64  
 1   an                int64  
 2   mois              int64  
 3   jour              int64  
 4   hrmn              object 
 5   lum               int64  
 6   agg               int64  
 7   int               int64  
 8   atm               float64
 9   col               float64
 10  com               object 
 11  adr               object 
 12  gps               object 
 13  lat               object 
 14  long              object 
 15  dep               object 
 16  catr              float64
 17  voie              object 
 18  v1                float64
 19  v2                object 
 20  circ              float64
 21  nbv               object 
 22  pr                object 
 23  pr1               object 
 24  vosp              float64
 25  prof              float64
 26  plan          

We need to adjust some column types

In [9]:
categorical_columns = [
    "lum", "agg", "int", "atm", "col", "catr", "circ", "prof", "plan", "surf",
    "infra", "situ", "env1", "senc", "catv", "catu", "grav", "sexe", "trajet",
    "secu", "locp", "etatp", "vosp", "obs", "obsm", "choc", "manv", "motor",
    "secu1", "secu2", "secu3", "place", "nbv", "actp", "pr", "pr1"
]

string_columns = [
    "com", "dep", "voie", "v2", "adr", "larrout",
    "lat", "long", "gps", "num_veh", "num_veh_usag", "id_vehicule",
    "id_vehicule_usag", "id_usager"
]

int_columns = ["an", "mois", "jour", "an_nais", "vma", "num_acc"]
time_columns = ["hrmn"]

# Apply conversions
for col in categorical_columns:
    if col in merged_df.columns:
        merged_df[col] = merged_df[col].astype("category")

for col in string_columns:
    if col in merged_df.columns:
        merged_df[col] = merged_df[col].astype("string")

for col in int_columns:
    if col in merged_df.columns:
        merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').astype("Int64")

for col in time_columns:
    if col in merged_df.columns:
        merged_df[col] = merged_df[col].astype(str)

# Convert lartpc to float
if "lartpc" in merged_df.columns:
    merged_df["lartpc"] = pd.to_numeric(merged_df["lartpc"], errors="coerce")

Adjust daytime

In [23]:
import re

# Define the cleaning function for hrmn
def clean_hrmn(val):
    if pd.isna(val):
        return "0000"
    
    val = str(val).strip()
    
    # Format: HH:MM
    if re.match(r"^\d{1,2}:\d{2}$", val):
        return val.replace(":", "").zfill(4)
    
    # Format: HHMM
    if val.isdigit() and len(val) in [3, 4]:
        return val.zfill(4)
    
    # Minute-only entry like "45" -> "0045"
    if val.isdigit() and len(val) <= 2:
        return "00" + val.zfill(2)
    
    # Invalid or unrecognized format
    return "0000"

# Apply cleaning
merged_df["hrmn_clean"] = merged_df["hrmn"].apply(clean_hrmn)

# Create datetime
merged_df["datetime"] = pd.to_datetime(
    merged_df["an"].astype(str) + "-" +
    merged_df["mois"].astype(str).str.zfill(2) + "-" +
    merged_df["jour"].astype(str).str.zfill(2) + " " +
    merged_df["hrmn_clean"].str[:2] + ":" + merged_df["hrmn_clean"].str[2:],
    format="%Y-%m-%d %H:%M",
    errors="coerce"
)

# Create time-related features
merged_df["hour"] = merged_df["datetime"].dt.hour.astype("Int64")
merged_df["dayofweek"] = merged_df["datetime"].dt.dayofweek.astype("Int64")
merged_df["weekday_name"] = merged_df["datetime"].dt.day_name().astype("category")

Unique value counts per year

In [32]:
print(merged_df['an'].value_counts())

an
2005    374561
2006    362507
2007    356228
2008    322196
2009    311706
2023    309341
2010    288112
2011    281675
2012    263194
2017    260392
2016    257286
2019    253488
2014    248642
2018    248406
2021    248187
2015    245706
2013    242163
2022    241487
2020    198907
Name: count, dtype: Int64


In [34]:
merged_df.describe()

,num_acc,an,mois,jour,v1,lartpc,occutc,an_nais,vma,datetime,hour,dayofweek
count,5314184.0,5314184.0,5314184.0,5314184.0,2.632973e+06,3.818966e+06,4.076093e+06,5292557.0,1251410.0,5314184,5314184.0,5314184.0
mean,201332794431.786469,2013.327611,6.701324,15.588233,-1.543806e-02,6.173533e+00,1.438132e-01,1975.503995,61.816122,2013-11-04 06:06:51.580249344,13.730389,2.984137
min,200500000001.0,2005.0,1.0,1.0,-1.000000e+00,0.000000e+00,0.000000e+00,1896.0,-1.0,2005-01-01 00:01:00,0.0,0.0
25%,200800056274.0,2008.0,4.0,8.0,0.000000e+00,0.000000e+00,0.000000e+00,1963.0,50.0,2008-09-28 17:13:00,10.0,1.0
50%,201300022821.0,2013.0,7.0,16.0,0.000000e+00,0.000000e+00,0.000000e+00,1978.0,50.0,2013-06-11 22:00:00,15.0,3.0
75%,201800041017.0,2018.0,10.0,23.0,0.000000e+00,0.000000e+00,0.000000e+00,1989.0,80.0,2018-09-15 16:30:00,18.0,5.0
max,202300054822.0,2023.0,12.0,31.0,9.000000e+00,9.600000e+02,9.000000e+02,2023.0,901.0,2023-12-31 23:50:00,23.0,6.0
std,565902720.582649,5.65908,3.374824,8.759571,5.852336e-01,2.375976e+01,3.074271e+00,18.55589,26.977133,NaN,5.267473,1.937736


Remove full row duplicates

In [36]:
print(merged_df.duplicated().sum())
merged_df = merged_df.drop_duplicates()

4644


Check the data balance and calculate the proportion of the most common value per column. Flag as "Unbalanced" if that value exceeds 70%.

In [41]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
balance_all_df = pd.DataFrame([
    {
        "Column": col,
        "Most Frequent Value %": round((vc := merged_df[col].value_counts(normalize=True, dropna=False)).iloc[0] * 100, 2),
        "Balance": "Unbalanced" if vc.iloc[0] > 0.7 else "Balanced",
        "Most Frequent Value": vc.idxmax() if vc.iloc[0] > 0.7 else None
    }
    for col in merged_df.columns
    if not merged_df[col].value_counts(normalize=True, dropna=False).empty
])
print(balance_all_df)

              Column  Most Frequent Value %     Balance  Most Frequent Value
0            num_acc                   0.09    Balanced                  NaN
1                 an                   7.05    Balanced                  NaN
2               mois                   9.48    Balanced                  NaN
3               jour                   3.44    Balanced                  NaN
4               hrmn                   1.26    Balanced                  NaN
5                lum                  69.61    Balanced                  NaN
6                agg                  61.65    Balanced                  NaN
7                int                  68.14    Balanced                  NaN
8                atm                  80.52  Unbalanced                  1.0
9                col                  32.45    Balanced                  NaN
10               com                   2.59    Balanced                  NaN
11               adr                  12.95    Balanced                  NaN

Check the missing data

In [39]:
# Replace actual NA values and string 'nan' with np.nan
merged_df = merged_df.apply(lambda col: col.map(lambda x: np.nan if pd.isna(x) or str(x).strip().lower() == "nan" else x))

missing_percent = merged_df.isnull().mean().multiply(100).round(2)
print(missing_percent)

num_acc              0.00
an                   0.00
mois                 0.00
jour                 0.00
hrmn                 0.00
lum                  0.00
agg                  0.00
int                  0.00
atm                  0.00
col                  0.00
com                  0.00
adr                 12.95
gps                 60.72
lat                 37.94
long                37.94
dep                  0.00
catr                 0.00
voie                 8.67
v1                  50.45
v2                  94.63
circ                 0.12
nbv                  0.22
pr                  34.40
pr1                 34.54
vosp                 0.22
prof                 0.15
plan                 0.19
lartpc              28.15
larrout              9.22
surf                 0.15
infra                0.46
situ                 0.42
env1                24.02
senc                 0.01
catv                 0.00
occutc              23.31
obs                  0.08
obsm                 0.04
choc        

Drop some categories with many NaN fields that cannot be used for the further analysis

In [48]:
merged_df.drop(columns=[c for c in ["v1", "v2", "pr", "pr1", "lartpc", "env1", "occutc", "vma",
                                    "id_vehicule", "motor", "id_vehicule_usag", "id_usager"]
                        if c in merged_df.columns], inplace=True)
print(merged_df.columns.tolist())

['num_acc', 'an', 'mois', 'jour', 'hrmn', 'lum', 'agg', 'int', 'atm', 'col', 'com', 'adr', 'gps', 'lat', 'long', 'dep', 'catr', 'voie', 'circ', 'nbv', 'vosp', 'prof', 'plan', 'larrout', 'surf', 'infra', 'situ', 'senc', 'catv', 'obs', 'obsm', 'choc', 'manv', 'num_veh', 'place', 'catu', 'grav', 'sexe', 'trajet', 'secu', 'locp', 'actp', 'etatp', 'an_nais', 'num_veh_usag', 'secu1', 'secu2', 'secu3', 'datetime', 'hour', 'dayofweek', 'weekday_name', 'hrmn_clean']


Statistical relationships with the target variable grav (injury severity)

In [50]:
# 1. Correlation with numeric columns
numeric_cols = [col for col in merged_df.select_dtypes(include=["number"]).columns if col != "grav"]
correlations = merged_df[numeric_cols].corrwith(merged_df["grav"]).dropna()
correlations = correlations.abs().sort_values(ascending=False)

print("Top numeric correlations with grav:\n")
print(correlations)

# 2. ANOVA F-test for categorical columns
from sklearn.preprocessing import LabelEncoder

categorical_cols = merged_df.select_dtypes(include=["category", "object", "string"]).columns
anova_scores = {}
for col in categorical_cols:
    if merged_df[col].nunique() > 1:
        try:
            X = LabelEncoder().fit_transform(merged_df[col].astype(str))
            y = merged_df["grav"]
            f_val, p_val = f_classif(X.reshape(-1, 1), y)
            anova_scores[col] = (f_val[0], p_val[0])
        except:
            continue

anova_df = pd.DataFrame(anova_scores, index=["F", "p"]).T.sort_values("F", ascending=False)
anova_df = anova_df[anova_df["p"] < 0.05]

print("\n Categorical features significantly related to grav (p < 0.05):")
print(anova_df)

Top numeric correlations with grav:

an_nais      0.100071
hour         0.013353
dayofweek    0.005026
mois         0.002354
num_acc      0.001466
an           0.001466
jour         0.000906
dtype: float64


C:\Users\jl\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw



 Categorical features significantly related to grav (p < 0.05):
                         F              p
grav                   inf   0.000000e+00
catu          6.131109e+04   0.000000e+00
gps           4.881921e+04   0.000000e+00
lat           4.079111e+04   0.000000e+00
agg           3.945956e+04   0.000000e+00
adr           3.298174e+04   0.000000e+00
place         3.033612e+04   0.000000e+00
obs           2.487901e+04   0.000000e+00
long          2.286869e+04   0.000000e+00
sexe          1.877374e+04   0.000000e+00
secu          1.659888e+04   0.000000e+00
nbv           1.454244e+04   0.000000e+00
plan          1.279565e+04   0.000000e+00
obsm          1.273760e+04   0.000000e+00
larrout       9.843337e+03   0.000000e+00
num_veh_usag  9.443238e+03   0.000000e+00
voie          9.006817e+03   0.000000e+00
dep           8.081652e+03   0.000000e+00
secu3         6.717772e+03   0.000000e+00
manv          6.533055e+03   0.000000e+00
secu1         6.488565e+03   0.000000e+00
etatp      

In [51]:
#extract processed csv
output_path = "../data/processed/accidents_processed.csv"

# Ensure the output directory exists
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Save to CSV
merged_df.to_csv(output_path, index=False)

print(f"✅ Processed data saved to: {output_path}")

✅ Processed data saved to: ../data/processed/accidents_processed.csv
